# Simple iteration for systems of linear equations

First, generate a random diagonally dominant matrix, for testing.

In [ ]:
import numpy as np
rndm = np.random.RandomState(1234)

n = 10
A = rndm.uniform(size=(n, n)) + np.diagflat([15]*n)
b = rndm.uniform(size=n)

# I.  Jacobi iteration

Given

$$
A x = b
$$

separate the diagonal part $D$,

$$ A = D + (A - D) $$

and write

$$
x = D^{-1} (D - A) x + D^{-1} b\;.
$$

Then iterate

$$
x_{n + 1} = B x_{n} + c\;,
$$

where

$$
B = D^{-1} (A - D) \qquad \text{and} \qquad c = D^{-1} b
$$


Let's construct the matrix and the r.h.s. for the Jacobi iteration

In [ ]:
diag_1d = np.diag(A)

B = -A.copy()
np.fill_diagonal(B, 0)

D = np.diag(diag_1d)
invD = np.diag(1./diag_1d)
BB = invD @ B
c = invD @ b

In [ ]:
# sanity checks
from numpy.testing import assert_allclose

assert_allclose(-B + D, A)


# xx is a "ground truth" solution, compute it using a direct method
xx = np.linalg.solve(A, b)

np.testing.assert_allclose(A@xx, b)
np.testing.assert_allclose(D@xx, B@xx + b)
np.testing.assert_allclose(xx, BB@xx + c)

Check that $\| B\| \leqslant 1$:

In [ ]:
np.linalg.norm(BB)

0.36436161983015336

### Do the Jacobi iteration

In [ ]:
n_iter = 50

x0 = np.ones(n)
x = x0
for _ in range(n_iter):
    x = BB @ x + c

In [ ]:
# Check the result:

A @ x - b

array([  1.11022302e-16,   1.11022302e-16,  -1.11022302e-16,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
        -1.38777878e-17,   0.00000000e+00,   2.77555756e-17,
         1.11022302e-16])

### Task I.1

Collect the proof-of-concept above into a single function implementing the Jacobi iteration. This function should receive the r.h.s. matrix $A$, the l.h.s. vector `b`, and the number of iterations to perform.


The matrix $A$ in the illustration above is strongly diagonally dominant, by construction.
What happens if the diagonal matrix elements of $A$ are made smaller? Check the convergence of the Jacobi iteration, and check the value of the norm of $B$.

(20% of the total grade)


In [2]:
import numpy as np

def jacobi_iteration(A, b, num_iters, x0=None, verbose=False):
    n = A.shape[0]
    D = np.diag(np.diag(A))
    D_inv = np.diag(1.0 / np.diag(A))
    B = D_inv @ (A - D)
    c = D_inv @ b

    # Initial guess
    x = np.zeros_like(b) if x0 is None else x0.copy()
    history = [x.copy()]

    # Spectral norm for convergence check
    B_norm = np.linalg.norm(B, 2)
    if verbose:
        print(f"Spectral norm of B: {B_norm:.6f}")

    for i in range(num_iters):
        x_new = B @ x + c
        history.append(x_new.copy())
        if verbose:
            print(f"Iter {i+1}, ||x_new - x|| = {np.linalg.norm(x_new-x):.6e}")
        x = x_new

    return x, history, B_norm

if __name__ == "__main__":
    A = np.array([[10, 1, 1],
                  [2, 10, 1],
                  [2, 2, 10]], dtype=float)
    b = np.array([12, 13, 14], dtype=float)

    x, hist, Bnorm = jacobi_iteration(A, b, num_iters=25, verbose=True)
    print("Final solution:", x)
    print("Spectral norm of B:", Bnorm)


    A2 = np.array([[2, 1, 1],
                   [2, 2, 1],
                   [2, 2, 2]], dtype=float)
    x2, hist2, Bnorm2 = jacobi_iteration(A2, b, num_iters=25, verbose=True)
    print("Final solution for weakly dominant A:", x2)
    print("Spectral norm of B (weakly dominant):", Bnorm2)

Spectral norm of B: 0.336147
Iter 1, ||x_new - x|| = 2.256103e+00
Iter 2, ||x_new - x|| = 6.835934e-01
Iter 3, ||x_new - x|| = 1.883083e-01
Iter 4, ||x_new - x|| = 5.439191e-02
Iter 5, ||x_new - x|| = 1.538962e-02
Iter 6, ||x_new - x|| = 4.392639e-03
Iter 7, ||x_new - x|| = 1.249677e-03
Iter 8, ||x_new - x|| = 3.558922e-04
Iter 9, ||x_new - x|| = 1.013362e-04
Iter 10, ||x_new - x|| = 2.885157e-05
Iter 11, ||x_new - x|| = 8.215520e-06
Iter 12, ||x_new - x|| = 2.339111e-06
Iter 13, ||x_new - x|| = 6.660406e-07
Iter 14, ||x_new - x|| = 1.896398e-07
Iter 15, ||x_new - x|| = 5.399710e-08
Iter 16, ||x_new - x|| = 1.537463e-08
Iter 17, ||x_new - x|| = 4.377665e-09
Iter 18, ||x_new - x|| = 1.246460e-09
Iter 19, ||x_new - x|| = 3.549077e-10
Iter 20, ||x_new - x|| = 1.010537e-10
Iter 21, ||x_new - x|| = 2.877307e-11
Iter 22, ||x_new - x|| = 8.192930e-12
Iter 23, ||x_new - x|| = 2.332454e-12
Iter 24, ||x_new - x|| = 6.642908e-13
Iter 25, ||x_new - x|| = 1.892348e-13
Final solution: [1.59100642 1.

# II. Seidel's iteration.

##### Task II.1

Implement the Seidel's iteration.

Test it on a random matrix. Study the convergence of iterations, relate to the norm of the iteration matrix.

(30% of the total grade)

In [4]:
import numpy as np

def gauss_seidel_iteration(A, b, num_iters, x0=None, verbose=False):
    n = A.shape[0]
    L = np.tril(A)
    U = A - L

    # Iteration matrix for GS: G = -L^{-1} U
    G = -np.linalg.inv(L) @ U
    GS_norm = np.linalg.norm(G, 2)

    # Initial guess
    x = np.zeros_like(b) if x0 is None else x0.copy()
    history = [x.copy()]

    if verbose:
        print(f"Spectral norm of Gauss-Seidel matrix G: {GS_norm:.6f}")

    for k in range(num_iters):
        x_new = x.copy()
        for i in range(n):
            sum1 = np.dot(A[i, :i], x_new[:i])
            sum2 = np.dot(A[i, i+1:], x[i+1:])
            x_new[i] = (b[i] - sum1 - sum2) / A[i, i]
        history.append(x_new.copy())
        if verbose:
            print(f"Iter {k+1}, ||x_new - x|| = {np.linalg.norm(x_new-x):.6e}")
        x = x_new

    return x, history, GS_norm

if __name__ == "__main__":
    # Test on a random diagonally dominant matrix
    np.random.seed(42)
    n = 5
    A = np.random.rand(n, n)

    for i in range(n):
        A[i, i] = np.sum(np.abs(A[i])) + 1
    b = np.random.rand(n)

    x_gs, hist_gs, norm_gs = gauss_seidel_iteration(A, b, num_iters=25, verbose=True)
    print("Final solution by Gauss-Seidel:", x_gs)
    print("Spectral norm of Gauss-Seidel iteration matrix:", norm_gs)


    A2 = np.random.rand(n, n)
    b2 = np.random.rand(n)
    x_gs2, hist_gs2, norm_gs2 = gauss_seidel_iteration(A2, b2, num_iters=25, verbose=True)
    print("Final solution for weakly dominant A:", x_gs2)
    print("Spectral norm of GS iteration matrix (weakly dominant):", norm_gs2)

Spectral norm of Gauss-Seidel matrix G: 0.466025
Iter 1, ||x_new - x|| = 3.141975e-01
Iter 2, ||x_new - x|| = 8.395942e-02
Iter 3, ||x_new - x|| = 1.137142e-02
Iter 4, ||x_new - x|| = 2.162794e-03
Iter 5, ||x_new - x|| = 1.070040e-04
Iter 6, ||x_new - x|| = 3.070953e-05
Iter 7, ||x_new - x|| = 2.057188e-06
Iter 8, ||x_new - x|| = 5.418870e-07
Iter 9, ||x_new - x|| = 3.324390e-08
Iter 10, ||x_new - x|| = 9.116781e-09
Iter 11, ||x_new - x|| = 5.446550e-10
Iter 12, ||x_new - x|| = 1.532302e-10
Iter 13, ||x_new - x|| = 9.083147e-12
Iter 14, ||x_new - x|| = 2.586997e-12
Iter 15, ||x_new - x|| = 1.508861e-13
Iter 16, ||x_new - x|| = 4.352887e-14
Iter 17, ||x_new - x|| = 2.519889e-15
Iter 18, ||x_new - x|| = 7.452207e-16
Iter 19, ||x_new - x|| = 4.857226e-17
Iter 20, ||x_new - x|| = 3.469447e-17
Iter 21, ||x_new - x|| = 8.673617e-18
Iter 22, ||x_new - x|| = 0.000000e+00
Iter 23, ||x_new - x|| = 0.000000e+00
Iter 24, ||x_new - x|| = 0.000000e+00
Iter 25, ||x_new - x|| = 0.000000e+00
Final solu

# III. Minimum residual scheme

### Task III.1

Implement the $\textit{minimum residual}$ scheme: an explicit non-stationary method, where at each step you select the iteration parameter $\tau_n$ to minimize the residual $\mathbf{r}_{n+1}$ given $\mathbf{r}_n$. Test it on a random matrix, study the convergence to the solution, in terms of the norm of the residual and the deviation from the ground truth solution (which you can obtain using a direct method). Study how the iteration parameter $\tau_n$ changes as iterations progress.

(50% of the grade)

In [5]:
import numpy as np

def minimum_residual_scheme(A, b, num_iters, x0=None, verbose=False):
    n = A.shape[0]
    x = np.zeros_like(b) if x0 is None else x0.copy()
    res_history = []
    dev_history = []
    tau_history = []

    # Ground truth solution via direct method
    x_true = np.linalg.solve(A, b)

    for k in range(num_iters):
        r = b - A @ x
        Ar = A @ r
        tau = np.dot(Ar, r) / np.dot(Ar, Ar) if np.dot(Ar, Ar) != 0 else 0
        x_new = x + tau * r
        res_norm = np.linalg.norm(b - A @ x_new)
        dev_norm = np.linalg.norm(x_new - x_true)
        if verbose:
            print(f"Iter {k+1}: tau={tau:.6e}, residual={res_norm:.6e}, deviation={dev_norm:.6e}")
        x = x_new
        res_history.append(res_norm)
        dev_history.append(dev_norm)
        tau_history.append(tau)

    return x, res_history, dev_history, tau_history, x_true

if __name__ == "__main__":
    np.random.seed(42)
    n = 5
    A = np.random.rand(n, n)
    A = A.T @ A + n * np.eye(n)
    b = np.random.rand(n)

    x_minres, res_hist, dev_hist, tau_hist, x_true = minimum_residual_scheme(A, b, num_iters=25, verbose=True)
    print("Final solution (min residual):", x_minres)
    print("Ground truth solution:", x_true)
    print("Final residual norm:", res_hist[-1])
    print("Final deviation from ground truth:", dev_hist[-1])
    print("Tau history:", tau_hist)

Iter 1: tau=1.084260e-01, residual=3.282764e-01, deviation=5.915655e-02
Iter 2: tau=1.518140e-01, residual=9.692059e-02, deviation=1.398255e-02
Iter 3: tau=1.092730e-01, residual=2.934925e-02, deviation=5.378506e-03
Iter 4: tau=1.528470e-01, residual=8.945052e-03, deviation=1.306683e-03
Iter 5: tau=1.096479e-01, residual=2.757874e-03, deviation=5.099503e-04
Iter 6: tau=1.534087e-01, residual=8.537393e-04, deviation=1.257405e-04
Iter 7: tau=1.098876e-01, residual=2.662712e-04, deviation=4.953912e-05
Iter 8: tau=1.538014e-01, residual=8.328277e-05, deviation=1.233668e-05
Iter 9: tau=1.100563e-01, residual=2.618296e-05, deviation=4.891863e-06
Iter 10: tau=1.540719e-01, residual=8.247039e-06, deviation=1.226250e-06
Iter 11: tau=1.101682e-01, residual=2.606293e-06, deviation=4.882638e-07
Iter 12: tau=1.542455e-01, residual=8.246241e-07, deviation=1.229001e-07
Iter 13: tau=1.102379e-01, residual=2.614427e-07, deviation=4.905996e-08
Iter 14: tau=1.543516e-01, residual=8.294736e-08, deviation=